# 🔍 Семантический поиск заведений по отзывам

В этом notebook мы:
1. Загружаем модель для создания эмбеддингов (sentence embeddings)
2. Создаем векторные представления всех отзывов
3. Строим FAISS индекс для быстрого поиска
4. Реализуем семантический поиск заведений
5. Тестируем на разных запросах
6. Визуализируем результаты

## 📦 Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from pathlib import Path
from typing import List, Tuple
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# Настройки визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Библиотеки загружены")

/home/keosido/.virtualenvs/2gis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Библиотеки загружены


## 🎯 Выбор модели для эмбеддингов

Доступные модели:
- **LaBSE** - 109 языков, универсальная (рекомендуется)
- **rubert-base-cased-sentence** - только русский, быстрая
- **sbert_large_nlu_ru** - русский, высокое качество

In [ ]:
# Выберите модель
# MODEL_NAME = 'sentence-transformers/LaBSE'  # Рекомендуется
MODEL_NAME = 'DeepPavlov/rubert-base-cased-sentence'  # Быстрая
# MODEL_NAME = 'ai-forever/sbert_large_nlu_ru'  # Качественная

print(f"📌 Выбрана модель: {MODEL_NAME}")
print(f"\n⏳ Загрузка модели (может занять несколько минут при первом запуске)...")

model = SentenceTransformer(MODEL_NAME)

print(f"✅ Модель загружена!")
print(f"   Размерность эмбеддингов: {model.get_sentence_embedding_dimension()}")

📌 Выбрана модель: sentence-transformers/LaBSE

⏳ Загрузка модели (может занять несколько минут при первом запуске)...


Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

## 📂 Загрузка данных

In [ ]:
# Загрузка отзывов (используем очищенные данные)
REVIEWS_FILE = '../nlp_analysis/reviews_cleaned_final.csv'  # Из предыдущего notebook
# Если очищенные данные нет, используйте:
# REVIEWS_FILE = 'reviews_full.csv'

PLACES_FILE = 'places.csv'

print(f"📂 Загрузка отзывов из {REVIEWS_FILE}...")
reviews_df = pd.read_csv(REVIEWS_FILE)

print(f"📂 Загрузка заведений из {PLACES_FILE}...")
places_df = pd.read_csv(PLACES_FILE)

print(f"\n✅ Данные загружены:")
print(f"   Отзывов: {len(reviews_df):,}")
print(f"   Заведений: {len(places_df):,}")

In [ ]:
# Просмотр структуры данных
print("Структура отзывов:")
print(reviews_df.info())

print("\nПервые 3 отзыва:")
reviews_df.head(3)

In [ ]:
# Базовая подготовка данных
# Определяем колонку с текстом отзывов
if 'review_text_clean' in reviews_df.columns:
    TEXT_COLUMN = 'review_text_clean'
else:
    TEXT_COLUMN = 'review_text'

print(f"📝 Используем колонку: {TEXT_COLUMN}")

# Заполняем пропуски и фильтруем короткие отзывы
reviews_df[TEXT_COLUMN] = reviews_df[TEXT_COLUMN].fillna('')
reviews_df = reviews_df[reviews_df[TEXT_COLUMN].str.len() > 10]

print(f"✅ После фильтрации: {len(reviews_df):,} отзывов")

## 🧮 Создание эмбеддингов

Это самый долгий шаг. Эмбеддинги сохраняются в кэш для быстрой загрузки в будущем.

In [ ]:
# Настройки
CACHE_PATH = 'cache/embeddings.pkl'
BATCH_SIZE = 32  # Уменьшите, если мало памяти

# Проверка кэша
if Path(CACHE_PATH).exists():
    print(f"✨ Найден кэш эмбеддингов: {CACHE_PATH}")
    print(f"   Загружаем из кэша...")
    
    with open(CACHE_PATH, 'rb') as f:
        embeddings = pickle.load(f)
    
    print(f"✅ Загружено {len(embeddings):,} эмбеддингов из кэша")
    print(f"   Размерность: {embeddings.shape}")
    
else:
    print(f"🔄 Кэш не найден. Создаем эмбеддинги...")
    print(f"⏱️  Это может занять 5-15 минут в зависимости от объема данных\n")
    
    # Получаем тексты отзывов
    reviews_text = reviews_df[TEXT_COLUMN].tolist()
    
    # Создание эмбеддингов
    embeddings = model.encode(
        reviews_text,
        batch_size=BATCH_SIZE,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    # Сохранение в кэш
    Path(CACHE_PATH).parent.mkdir(parents=True, exist_ok=True)
    with open(CACHE_PATH, 'wb') as f:
        pickle.dump(embeddings, f)
    
    print(f"\n✅ Создано {len(embeddings):,} эмбеддингов")
    print(f"   Размерность: {embeddings.shape}")
    print(f"   Сохранено в кэш: {CACHE_PATH}")

In [ ]:
# Проверка размерности
print(f"Размер эмбеддингов в памяти: {embeddings.nbytes / (1024**2):.2f} MB")
print(f"Форма массива: {embeddings.shape}")
print(f"Тип данных: {embeddings.dtype}")

## 🏗️ Построение FAISS индекса

FAISS - библиотека от Facebook для быстрого поиска ближайших соседей.

In [ ]:
print("🏗️ Построение FAISS индекса...\n")

dimension = embeddings.shape[1]
num_vectors = len(embeddings)

print(f"Размерность векторов: {dimension}")
print(f"Количество векторов: {num_vectors:,}")

# Выбор типа индекса
if num_vectors < 100000:
    # Для небольших датасетов - простой индекс (точный поиск)
    print("\nИспользуем IndexFlatIP (Inner Product) - точный поиск")
    index = faiss.IndexFlatIP(dimension)
else:
    # Для больших датасетов - индекс с квантизацией (быстрый поиск)
    print("\nИспользуем IndexIVFFlat - быстрый приближенный поиск")
    nlist = 100  # количество кластеров
    quantizer = faiss.IndexFlatIP(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
    
    print("Обучение индекса...")
    index.train(embeddings)

# Нормализация для косинусного сходства
print("Нормализация векторов...")
faiss.normalize_L2(embeddings)

# Добавление векторов в индекс
print("Добавление векторов в индекс...")
index.add(embeddings)

print(f"\n✅ Индекс построен!")
print(f"   Векторов в индексе: {index.ntotal:,}")

## 🔍 Функции поиска

In [ ]:
def search_reviews(query: str, top_k: int = 20) -> pd.DataFrame:
    """
    Поиск релевантных отзывов по запросу
    
    Args:
        query: Текстовый запрос
        top_k: Количество результатов
    
    Returns:
        DataFrame с релевантными отзывами
    """
    # Создаем эмбеддинг запроса
    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)
    
    # Поиск
    distances, indices = index.search(query_embedding, top_k)
    
    # Формируем результат
    results = reviews_df.iloc[indices[0]].copy()
    results['similarity_score'] = distances[0]
    
    return results


def search_places(query: str, top_k: int = 10, 
                 min_reviews: int = 3, 
                 aggregation: str = 'weighted') -> pd.DataFrame:
    """
    Поиск заведений по описательному запросу
    
    Args:
        query: Запрос (например, "уютное кафе с вкусным кофе")
        top_k: Количество заведений
        min_reviews: Минимум релевантных отзывов
        aggregation: Метод агрегации ('mean', 'max', 'weighted')
    
    Returns:
        DataFrame с топ заведениями
    """
    # Находим релевантные отзывы
    relevant_reviews = search_reviews(query, top_k=200)
    
    # Группируем по заведениям
    place_scores = relevant_reviews.groupby('place_id').agg({
        'similarity_score': ['mean', 'max', 'count']
    }).reset_index()
    
    place_scores.columns = ['place_id', 'avg_score', 'max_score', 'review_count']
    
    # Фильтруем
    place_scores = place_scores[place_scores['review_count'] >= min_reviews]
    
    # Агрегация
    if aggregation == 'mean':
        place_scores['final_score'] = place_scores['avg_score']
    elif aggregation == 'max':
        place_scores['final_score'] = place_scores['max_score']
    elif aggregation == 'weighted':
        place_scores['final_score'] = (
            place_scores['avg_score'] * np.log1p(place_scores['review_count'])
        )
    
    # Топ заведения
    place_scores = place_scores.nlargest(top_k, 'final_score')
    
    # Добавляем информацию о заведениях
    results = place_scores.merge(
        places_df,
        left_on='place_id',
        right_on='id',
        how='left'
    )
    
    return results[['name', 'address', 'category', 'rating',
                   'final_score', 'review_count', 'place_id']]


def get_place_highlights(place_id: int, query: str, top_k: int = 3) -> List[str]:
    """
    Получить самые релевантные отзывы для заведения
    """
    relevant_reviews = search_reviews(query, top_k=100)
    place_reviews = relevant_reviews[relevant_reviews['place_id'] == place_id]
    
    return place_reviews.nlargest(top_k, 'similarity_score')[TEXT_COLUMN].tolist()


print("✅ Функции поиска определены")

## 🧪 Тестирование поиска

Давайте протестируем систему на разных запросах!

In [ ]:
# Тест 1: Базовый запрос
query = "уютное кафе с вкусным кофе"

print(f"🔍 Запрос: '{query}'\n")
print("=" * 80)

results = search_places(query, top_k=10)

print(f"\nНайдено {len(results)} заведений:\n")
for idx, row in results.iterrows():
    print(f"{idx+1}. {row['name']}")
    print(f"   📍 {row['address']}")
    print(f"   🏷️  {row['category']}")
    print(f"   ⭐ Рейтинг: {row['rating']:.1f}")
    print(f"   🎯 Релевантность: {row['final_score']:.3f}")
    print(f"   💬 Релевантных отзывов: {int(row['review_count'])}")
    print()

In [ ]:
# Показываем релевантные отзывы для топ заведения
if len(results) > 0:
    top_place = results.iloc[0]
    
    print("=" * 80)
    print(f"💬 Релевантные отзывы для '{top_place['name']}'")
    print("=" * 80)
    
    highlights = get_place_highlights(top_place['place_id'], query, top_k=5)
    
    for i, review in enumerate(highlights, 1):
        print(f"\n{i}. {review[:300]}...")

In [ ]:
# Тест 2: Разные типы запросов
test_queries = [
    "недорогое кафе с большими порциями",
    "романтический ресторан для свидания",
    "тихое место для работы с wi-fi",
    "семейное кафе с детской зоной",
    "модный бар с хорошими коктейлями"
]

print("🧪 Тестирование разных запросов\n")
print("=" * 80)

for query in test_queries:
    print(f"\n🔍 {query}")
    results = search_places(query, top_k=3)
    
    for idx, row in results.iterrows():
        print(f"   {idx+1}. {row['name']} - ⭐{row['rating']:.1f} (релевантность: {row['final_score']:.3f})")
    
    print("-" * 80)

## 📊 Визуализация результатов

In [ ]:
# Визуализация для одного запроса
query = "уютное кафе с вкусным кофе"
results = search_places(query, top_k=15)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# График 1: Рейтинг vs Релевантность
ax1 = axes[0]
scatter = ax1.scatter(
    results['rating'], 
    results['final_score'],
    s=results['review_count'] * 20,  # Размер = кол-во отзывов
    c=results['final_score'],
    cmap='viridis',
    alpha=0.6,
    edgecolors='black'
)
ax1.set_xlabel('Рейтинг 2GIS', fontsize=12)
ax1.set_ylabel('Релевантность запросу', fontsize=12)
ax1.set_title('Рейтинг vs Релевантность', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax1, label='Релевантность')

# График 2: Топ заведения
ax2 = axes[1]
top_10 = results.head(10)
y_pos = np.arange(len(top_10))
bars = ax2.barh(y_pos, top_10['final_score'], color='skyblue', edgecolor='black')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([name[:30] for name in top_10['name']], fontsize=9)
ax2.set_xlabel('Релевантность', fontsize=12)
ax2.set_title(f'Топ-10 заведений\nЗапрос: "{query}"', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"💡 Размер точки = количество релевантных отзывов")

In [ ]:
# Сравнение нескольких запросов
queries = [
    "уютное кафе",
    "дешевое кафе",
    "романтический ресторан"
]

fig, axes = plt.subplots(1, len(queries), figsize=(18, 5))

for idx, query in enumerate(queries):
    results = search_places(query, top_k=10)
    
    ax = axes[idx]
    top_5 = results.head(5)
    
    y_pos = np.arange(len(top_5))
    ax.barh(y_pos, top_5['final_score'], color=plt.cm.Set3(idx))
    ax.set_yticks(y_pos)
    ax.set_yticklabels([name[:25] for name in top_5['name']], fontsize=8)
    ax.set_xlabel('Релевантность')
    ax.set_title(f'"{query}"', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Интерактивный поиск

Попробуйте свои запросы!

In [ ]:
# Введите свой запрос
my_query = "место с хорошей музыкой и атмосферой"  # ← Измените здесь

print(f"🔍 Ваш запрос: '{my_query}'\n")
print("=" * 80)

results = search_places(my_query, top_k=10)

for idx, row in results.iterrows():
    print(f"\n{idx+1}. {row['name']}")
    print(f"   📍 {row['address']}")
    print(f"   ⭐ {row['rating']:.1f} | 🎯 {row['final_score']:.3f} | 💬 {int(row['review_count'])} отзывов")
    
    # Показываем один релевантный отзыв
    highlights = get_place_highlights(row['place_id'], my_query, top_k=1)
    if highlights:
        print(f"   💭 \"{highlights[0][:150]}...\"")

## 📈 Анализ качества поиска

In [ ]:
# Статистика по similarity scores
query = "уютное кафе с вкусным кофе"
relevant_reviews = search_reviews(query, top_k=100)

print("=" * 80)
print("АНАЛИЗ КАЧЕСТВА ПОИСКА")
print("=" * 80)

print(f"\nЗапрос: '{query}'")
print(f"\nСтатистика similarity scores (топ-100 отзывов):")
print(relevant_reviews['similarity_score'].describe())

# Гистограмма similarity scores
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(relevant_reviews['similarity_score'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Similarity Score')
plt.ylabel('Количество')
plt.title('Распределение similarity scores', fontweight='bold')
plt.axvline(relevant_reviews['similarity_score'].mean(), color='red', 
           linestyle='--', label=f'Среднее: {relevant_reviews["similarity_score"].mean():.3f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(len(relevant_reviews)), 
        relevant_reviews['similarity_score'].values, 
        marker='o', markersize=3)
plt.xlabel('Ранг')
plt.ylabel('Similarity Score')
plt.title('Убывание релевантности', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 💾 Сохранение модели и индекса

In [ ]:
# Сохранение FAISS индекса для быстрой загрузки в будущем
INDEX_PATH = 'cache/faiss_index.bin'

Path(INDEX_PATH).parent.mkdir(parents=True, exist_ok=True)
faiss.write_index(index, INDEX_PATH)

print(f"✅ FAISS индекс сохранен в {INDEX_PATH}")
print(f"\nДля загрузки в будущем используйте:")
print(f"index = faiss.read_index('{INDEX_PATH}')")

## 🎓 Эксперименты и улучшения

Попробуйте:
1. Разные модели эмбеддингов
2. Разные методы агрегации ('mean', 'max', 'weighted')
3. Разные значения `min_reviews`
4. Комбинированный поиск (семантический + фильтры)

In [ ]:
# Сравнение методов агрегации
query = "уютное кафе"

print("Сравнение методов агрегации:\n")

for method in ['mean', 'max', 'weighted']:
    results = search_places(query, top_k=5, aggregation=method)
    
    print(f"\n{method.upper()}:")
    for idx, row in results.iterrows():
        print(f"  {idx+1}. {row['name'][:40]} - {row['final_score']:.3f}")

## ✅ Резюме

Вы успешно создали систему семантического поиска заведений!

**Что дальше:**
1. Запустите `app.py` для веб-интерфейса: `streamlit run app.py`
2. Изучите продвинутые техники в `03_advanced_analytics.ipynb`
3. Экспериментируйте с разными моделями и параметрами